In [59]:
# MM21B030

In [60]:
# all imports

import pandas as pd
import numpy as np
import os
import re
from string import punctuation
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import random
from sklearn.model_selection import KFold
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.utils import resample
from sklearn.base import clone


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hiran\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# loading data

data = []
import os

directory = 'C:/Users/hiran/Desktop/SEM7/FML/assignment2/enron1/ham'
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        try:
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                content = file.read()
                data.append({'X': content, 'y' : '0'})
        except UnicodeDecodeError:
            print(filename, "has a different encoding than utf-8")
            
directory = 'C:/Users/hiran/Desktop/SEM7/FML/assignment2/enron1/spam'
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        try:
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                content = file.read()
                data.append({'X': content, 'y' : '1'})
        except UnicodeDecodeError:
            print(filename, "has a different encoding than utf-8")

data = pd.DataFrame(data)
print(data.shape)

X_train, X_test, y_train, y_test = train_test_split(data['X'], data['y'], test_size=0.01, random_state=42)


0754.2004-04-01.GP.spam.txt has a different encoding than utf-8
1414.2004-06-24.GP.spam.txt has a different encoding than utf-8
2042.2004-08-30.GP.spam.txt has a different encoding than utf-8
2140.2004-09-13.GP.spam.txt has a different encoding than utf-8
2248.2004-09-23.GP.spam.txt has a different encoding than utf-8
2526.2004-10-17.GP.spam.txt has a different encoding than utf-8
2649.2004-10-27.GP.spam.txt has a different encoding than utf-8
2698.2004-10-31.GP.spam.txt has a different encoding than utf-8
3304.2004-12-26.GP.spam.txt has a different encoding than utf-8
3364.2005-01-01.GP.spam.txt has a different encoding than utf-8
4142.2005-03-31.GP.spam.txt has a different encoding than utf-8
4201.2005-04-05.GP.spam.txt has a different encoding than utf-8
4350.2005-04-23.GP.spam.txt has a different encoding than utf-8
4566.2005-05-24.GP.spam.txt has a different encoding than utf-8
5105.2005-08-31.GP.spam.txt has a different encoding than utf-8
(5157, 2)


Data preprocessing

1. Strip extra spaces and tabs
2. Strip spl characters
3. Replace numbers with 'number'
4. Strip english stop words
5. Replace links with 'link'
6. Lowercase all characters

In [63]:

def pre_processing(line):
    # 1. Strip extra spaces and tabs
    line = re.sub(r'\s+', ' ', line).strip()

    # 2. Strip special characters
    line = re.sub(r'[^\w\s]', '', line)  # Removes all characters except alphanumeric and whitespace

    # 3. Replace numbers with 'number'
    line = re.sub(r'\d+', 'number', line)

    # 4. Strip English stop words
    stop_words = set(stopwords.words('english'))
    line = ' '.join([word for word in line.split() if word.lower() not in stop_words])

    # 5. Replace links with 'link'
    line = re.sub(r'http\S+|www\S+|https\S+', 'link', line)

    # 6. Lowercase all characters
    line = line.lower()
    
    return line
    
# Example input string
line = "This is a sample line with   spaces, http://example.com, 123 numbers, and special characters !@#."

print(pre_processing(line))


    

sample line spaces link number numbers special characters


Implementation of Naive Bayes Algorithm

In [64]:

class NaiveBayes():
    def __init__(self):
        self.vectorizer = CountVectorizer()
        self.p_spam = None
        self.p_spam_words = None
        self.p_ham_words = None
    
    def fit(self, X, y):
        # Preprocess the training data
        X_processed = X.apply(pre_processing)
        
        # Vectorize the training data
        X_vectorized = self.vectorizer.fit_transform(X_processed)
        
        # Separate spam and ham messages
        X_spam = X_vectorized[y == '1']
        X_ham = X_vectorized[y == '0']
        
        # Calculate the probability of spam (P(spam))
        self.p_spam = X_spam.shape[0] / (X_spam.shape[0] + X_ham.shape[0])
        
        # Calculate the probability of each word given spam (P(word|spam)) and ham (P(word|ham))
        self.p_spam_words = (X_spam.sum(axis=0) + 1) / (X_spam.sum() + X_vectorized.shape[1])
        self.p_ham_words = (X_ham.sum(axis=0) + 1) / (X_ham.sum() + X_vectorized.shape[1])
        
        return self
    
    def predict(self, X_test):
        
        X_processed = X_test.apply(pre_processing)
        X_vectorized = self.vectorizer.transform(X_processed)
        
        def prediction(x_vec):
            x_vec = x_vec.toarray()
            # Calculate log probabilities for numerical stability
            log_prob_spam = np.log(self.p_spam) + x_vec.dot(np.log(self.p_spam_words.T)) + (1 - x_vec).dot(np.log(1 - self.p_spam_words.T))
            log_prob_ham = np.log(1 - self.p_spam) + x_vec.dot(np.log(self.p_ham_words.T)) + (1 - x_vec).dot(np.log(1 - self.p_ham_words.T))
            
            # Return 1 if spam, 0 if ham
            return '1' if log_prob_spam > log_prob_ham else '0'
            
        
        return np.array([prediction(x) for x in X_vectorized])
    
    def get_params(self, deep=True):
        return {}
    
    def set_params(self, **params):
        return self


# scores = cross_val_score(NaiveBayes(), X_train, y_train, cv=5, scoring='f1_macro')
# print(scores.mean())
        
    

Implementation of ensemble methods using boosting

In [6]:
X_train.shape[0]

5105

In [56]:
class weak_classifier():
    
    def __init__(self):
        self.vectorizer = CountVectorizer()
        self.p_spam = None
        self.p_spam_words = None
        self.p_ham_words = None
        self.cut_off = 0
        self.sep = 0
    
    def fit(self, X, y):
        
        # Find one word that is most propable in spam than ham
        
        X_processed = X.apply(pre_processing)
        
        # Vectorize the training data
        X_vectorized = self.vectorizer.fit_transform(X_processed)
        
        # Separate spam and ham messages
        X_spam = X_vectorized[y == '1']
        X_ham = X_vectorized[y == '0']
        
        # Calculate the probability of spam (P(spam))
        self.p_spam = X_spam.shape[0] / (X_spam.shape[0] + X_ham.shape[0])
        
        # Calculate the probability of each word given spam (P(word|spam)) and ham (P(word|ham))
        # Convert to lists
        self.p_spam_words = np.array((X_spam.sum(axis=0) + 1) / (X_spam.sum() + X_vectorized.shape[1])).flatten()
        self.p_ham_words = np.array((X_ham.sum(axis=0) + 1) / (X_ham.sum() + X_vectorized.shape[1])).flatten()
        # Calculate the "fishiness" ratio for each word
        fishiness_ratio = self.p_spam_words / self.p_ham_words

        # Get the indices of the top fishy words by sorting the ratios in descending order
        self.top_fishy_indices = np.argsort(fishiness_ratio)[-25:][::-1]  # Top 25 fishy words

        
        return self
        
    def predict(self, X_test):
        X_processed = X_test.apply(pre_processing)
        X_vectorized = self.vectorizer.transform(X_processed)
        
        def prediction(x_vec):
            x_vec = x_vec.toarray()[0]
            x_fishy = x_vec[self.top_fishy_indices]
        
            # Calculate the log probabilities for the top fishy words
            log_prob_spam = np.log(self.p_spam + 10e-10) + np.sum(np.log(self.p_spam_words[self.top_fishy_indices]) * x_fishy)
            log_prob_ham = np.log(1 - self.p_spam + 10e-10) + np.sum(np.log(self.p_ham_words[self.top_fishy_indices] + 10e-10) * x_fishy)
            
            # Return '1' for spam and '0' for ham based on the comparison
            return '1' if log_prob_spam > log_prob_ham else '0'
            
        
        return np.array([prediction(x) for x in X_vectorized])
    def print_fishy_words(self):
        print([(self.vectorizer.get_feature_names_out()[i]) for i in self.top_fishy_indices])
    def get_params(self, deep=True):
        return {}
    
    def set_params(self, **params):
        return self
    
import pickle
model = weak_classifier()
model.fit(X_train, y_train)
# Save the model to a file
with open('weakclassifier.pkl', 'wb') as file:
    pickle.dump(model, file)
model.print_fishy_words()    
scores = cross_val_score(weak_classifier(), X_train, y_train, cv=5, scoring='f1_macro')
print(scores) 


['td', 'nbsp', 'width', 'pills', 'computron', 'href', 'height', 'viagra', 'font', 'xp', 'src', 'cialis', 'soft', 'meds', 'paliourg', 'php', 'bgcolor', 'oo', 'voip', 'drugs', 'biz', 'mx', 'img', 'photoshop', 'valign']
[0.71420819 0.7146553  0.7100819  0.73973921 0.69256534]


In [57]:
class boosting():
    
    def __init__(self, n_classifiers=50):
        self.n_classifiers = n_classifiers
        self.vectorizer = CountVectorizer()
        self.weak_classifier = weak_classifier()
    
        self.alphas = []
        self.classifiers = []
    
    def fit(self, X, y):
        n_samples = X.shape[0]
        w = np.full(n_samples, (1 / n_samples))  # Initialize weights to 1/n_samples

        for t in range(self.n_classifiers):
            clf = clone(self.weak_classifier)
            
            # Resample based on sample weights
            probabilities = w / np.sum(w)

            # Resample indices based on sample weights
            n_samples = X.shape[0]
            resampled_indices = np.random.choice(np.arange(n_samples), size=n_samples, replace=True, p=probabilities)

            # Create resampled dataset
            X_resampled = X.iloc[resampled_indices]
            y_resampled = y.iloc[resampled_indices]
            clf.fit(X_resampled, y_resampled)
            
            # Make predictions
            predictions = clf.predict(X)

            # Compute weighted error rate
            error = 0
            for i in range(n_samples):
                error += (predictions[i] != y.iloc[i])
            
            error /= n_samples
            
            # Calculate alpha (classifier weight) based on the error
            alpha = 0.5 * np.log((1.0 - error) / error)

            # Update sample weights
            # Increase weights for misclassified samples, decrease for correctly classified
            for i in range(n_samples):
                w[i] *= np.exp(-alpha * (predictions[i] == y.iloc[i]))

            # Normalize weights so they sum to 1
            w /= np.sum(w)


            # Store classifier and alpha
            self.classifiers.append(clf)
            self.alphas.append(alpha)

        return self.classifiers, self.alphas  # Return all classifiers and alphas

    def predict(self, X):
        clf_preds = np.array([alpha * np.where(clf.predict(X) == '1', 1, -1) for clf, alpha in zip(self.classifiers, self.alphas)])
        # Sum across classifiers to get the final prediction score for each instance
        y_pred = np.sum(clf_preds, axis=0)
        return np.array(['1' if x > 0 else '0' for x in y_pred])
    
    def get_params(self, deep=True):
        return {}
    
    def set_params(self, **params):
        return self

import pickle
model = boosting()
model.fit(X_train, y_train)
# Save the model to a file
with open('boostingmodel.pkl', 'wb') as file:
    pickle.dump(model, file) 
print("Model Saved Successfully")   
scores = cross_val_score(boosting(), X_train, y_train, cv=5, scoring='f1_macro')
print(scores)  

 
        
        

Model Saved Successfully
[0.40948525 0.42021579 0.41422834 0.41889585 0.41756988]


In [58]:
print(accuracy_score(model.predict(X_train), y_train))

0.7126346718903036


In [62]:
# loading data

data = []
import os

directory = 'C:/Users/hiran/Desktop/SEM7/FML/assignment2/enron1/ham'
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        try:
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                content = file.read()
                data.append({'X': content, 'y' : '0'})
        except UnicodeDecodeError:
            print(filename, "has a different encoding than utf-8")
            
directory = 'C:/Users/hiran/Desktop/SEM7/FML/assignment2/enron1/spam'
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        try:
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                content = file.read()
                data.append({'X': content, 'y' : '1'})
        except UnicodeDecodeError:
            print(filename, "has a different encoding than utf-8")
            
directory = 'C:/Users/hiran/Desktop/SEM7/FML/assignment2/enron6/ham'
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        try:
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                content = file.read()
                data.append({'X': content, 'y' : '0'})
        except UnicodeDecodeError:
            print(filename, "has a different encoding than utf-8")
            
directory = 'C:/Users/hiran/Desktop/SEM7/FML/assignment2/enron6/spam'
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        try:
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                content = file.read()
                data.append({'X': content, 'y' : '1'})
        except UnicodeDecodeError:
            print(filename, "has a different encoding than utf-8")

data = pd.DataFrame(data)
print(data.shape)

X_train, X_test, y_train, y_test = train_test_split(data['X'], data['y'], test_size=0.01, random_state=42)

0754.2004-04-01.GP.spam.txt has a different encoding than utf-8
1414.2004-06-24.GP.spam.txt has a different encoding than utf-8
2042.2004-08-30.GP.spam.txt has a different encoding than utf-8
2140.2004-09-13.GP.spam.txt has a different encoding than utf-8
2248.2004-09-23.GP.spam.txt has a different encoding than utf-8
2526.2004-10-17.GP.spam.txt has a different encoding than utf-8
2649.2004-10-27.GP.spam.txt has a different encoding than utf-8
2698.2004-10-31.GP.spam.txt has a different encoding than utf-8
3304.2004-12-26.GP.spam.txt has a different encoding than utf-8
3364.2005-01-01.GP.spam.txt has a different encoding than utf-8
4142.2005-03-31.GP.spam.txt has a different encoding than utf-8
4201.2005-04-05.GP.spam.txt has a different encoding than utf-8
4350.2005-04-23.GP.spam.txt has a different encoding than utf-8
4566.2005-05-24.GP.spam.txt has a different encoding than utf-8
5105.2005-08-31.GP.spam.txt has a different encoding than utf-8
0003.2004-08-01.BG.spam.txt has a differ

In [65]:
model = NaiveBayes()
model.fit(X_train, y_train)
# Save the model to a file
with open('naivemodel.pkl', 'wb') as file:
    pickle.dump(model, file) 
print("Model Saved Successfully")   
scores = cross_val_score(boosting(), X_train, y_train, cv=5, scoring='f1_macro')
print(scores)  

Model Saved Successfully
[0.804271   0.78366472 0.78011044 0.76593481 0.74651994]
